This notebook is showing how to slice the expression level of a single gene in < 10ms. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
def loadDf(fname,mmap_mode='r'):
    with open(fname+'.index.txt') as f:
        myIndex=map(lambda s:s.replace("\n",""), f.readlines())
    with open(fname+'.columns.txt') as f:
        myColumns=map(lambda s:s.replace("\n",""), f.readlines())
    tmpMatrix=np.load(fname+".npy",mmap_mode=mmap_mode)
    tmpDf=pd.DataFrame(tmpMatrix,index=myIndex,columns=myColumns)
    tmpDf.columns.name='Run'
    return tmpDf

### Setting directory for expression matrix

In [3]:
baseDir='/home/jovyan/efs/all_seq/rnaseq_merged/' #Base directory

expression_metric='tpm' #offer Kallisto expression metric: ["tpm","est_counts"]
specie='Homo_sapiens' #Check files in baseDir directory for more species
data_matrix_dir=baseDir+'/{specie}.gene_symbol.{expression_metric}'.format(specie=specie,
                                            expression_metric=expression_metric)

In [6]:
#!ls /home/jovyan/efs/

In [4]:
#a memory mapped dataframe
rnaseqDf=loadDf(data_matrix_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/efs/all_seq/rnaseq_merged//Homo_sapiens.gene_symbol.tpm.index.txt'

### Layout of expression matrix

In [ ]:
print ('Dimension of expression matrix: (# of genes, # of sequencing run) ',rnaseqDf.shape)

In [ ]:
rnaseqDf.head()

### Query a single gene

In [ ]:
queryGene='TP53'

In [ ]:
%time hitGeneS=rnaseqDf.loc[queryGene]

In [ ]:
print ('# sequencing runs extracted: ',len(hitGeneS))

Distribution of expression over first 100 runs

In [ ]:
%matplotlib inline

#pandas sample is slow for some reason, use numpy choice instead. 
mySrrs=np.random.choice(hitGeneS.index,size=100)
tmpS=hitGeneS.loc[mySrrs]
tmpS.hist()